# Dependência

In [ ]:
# Install required Google Cloud packages (commented out as these are typically one-time setup commands)
# !pip install gcloud
# !gcloud auth application-default login

# Import necessary Python libraries
import pandas as pd                # Data manipulation and analysis
import numpy as np                 # Numerical computing
import time                        # Time-related functions
import os                          # Operating system interfaces
import pandas_gbq                  # Pandas integration with BigQuery
from google.cloud import bigquery  # BigQuery client library
import glob                        # File path pattern matching
import openpyxl                    # Excel file handling
import csv                         # CSV file handling
import re                          # Regular expressions

# Note: The actual imports remain exactly as in the original code

# Tratamento

In [ ]:
# Load Public Safety data from Excel file
df = pd.read_excel("Base_Estadic_2023(2).xlsx", sheet_name='Segurança Pública')

# Select and rename relevant columns for military police data
df = df[['Sigla UF', 'PopUF', 'ESEG271', 'ESEG272']]
df = df.rename(columns={
    'ESEG271': 'policia_militar_efetivo_homens',
    'ESEG272': 'policia_militar_efetivo_mulheres',
    'PopUF': 'pop_projetada',
    'Sigla UF': 'sigla_uf'
})

# Handle missing/refused data by replacing with 0
df['policia_militar_efetivo_homens'] = df['policia_militar_efetivo_homens'].replace(['Recusa', 'Não informou'], 0)
df['policia_militar_efetivo_mulheres'] = df['policia_militar_efetivo_mulheres'].replace(['Recusa', 'Não informou'], 0)

# Convert to numeric (note: these need assignment to take effect)
df['policia_militar_efetivo_homens'] = df['policia_militar_efetivo_homens'].astype(str).astype(int)
df['policia_militar_efetivo_mulheres'] = df['policia_militar_efetivo_mulheres'].astype(str).astype(int)

# Add year column
df['ano'] = 2023

# Create separate dataframes by gender
df_mulheres = df.drop('policia_militar_efetivo_homens', axis=1)
df_homens = df.drop('policia_militar_efetivo_mulheres', axis=1)

# Standardize column names and add gender column
df_homens = df_homens.rename(columns={'policia_militar_efetivo_homens': 'quantidade_pessoas'})
df_homens['genero'] = 'Masculino'

df_mulheres = df_mulheres.rename(columns={'policia_militar_efetivo_mulheres': 'quantidade_pessoas'})
df_mulheres['genero'] = 'Feminino'

# Combine male and female dataframes
df = pd.concat([df_homens, df_mulheres])

# Add theme identifier
df['tema'] = 'seguranca_publica'

# Upload

In [ ]:
# Define the BigQuery table schema with field descriptions (in Portuguese)
schema = [
    bigquery.SchemaField('ano', 'INTEGER', description='Ano de implementação da legislação.'),
    bigquery.SchemaField('tema', 'STRING', description='Tema de origem daquele dado na ESTADIC'),
    bigquery.SchemaField('sigla_uf', 'STRING', description='Sigla da Unidade da Federação.'),
    bigquery.SchemaField('quantidade_pessoas', 'INTEGER',
                       description='Quantitativo total de pessoas'),
    bigquery.SchemaField('genero', 'STRING',
                       description='Genero daquele quantitativo')
]

# Initialize BigQuery client connection
client = bigquery.Client(project='repositoriodedadosgpsp')

# Create reference to target dataset
dataset_ref = client.dataset('perfil_remuneracao')

# Create reference to target table with standardized naming convention:
# Format: FONTE_algo_intuitivo_dado (ESTADIC_policial_militar_genero_v1)
table_ref = dataset_ref.table('ESTADIC_policial_militar_genero_v1')

# Configure the load job with our schema definition
job_config = bigquery.LoadJobConfig(schema=schema)

# Execute the load job to transfer DataFrame to BigQuery
job = client.load_table_from_dataframe(
    dataframe=df,
    destination=table_ref,
    job_config=job_config
)

# Wait for the job to complete
job.result()